<a href="https://colab.research.google.com/github/vimalkum/Hindi-POS-Tagger/blob/master/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)
training = open("/content/drive/My Drive/Colab Notebooks/hin_health_set02.txt","r").read()
training=training.split('\n')
sentence,tags=[],[]
#print(len(training))
for i in range(0,len(training)-1):
  sen,tag=[],[]
  #print(training[i])
  for word in training[i].split()[1:]:
    w = word.split('//')
    sen.append(w[0])
    tag.append(w[1])
    #print(w[0],w[1])
  sentence.append(sen)
  tags.append(tag)

print(sentence)

training_data = []
for sent,tag in zip(sentence,tags):
  training_data.append((sent,tag))
                       

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {}
ix_to_tag={}
for sent, tags in training_data:
    for tag in tags:
        if tag not in tag_to_ix:
            l=len(tag_to_ix)
            tag_to_ix[tag] = l
            ix_to_tag[l]=tag
print(tag_to_ix)

[['गर्भाशय', 'की', 'रसौली', ',', 'फाइब्रायड', 'या', 'लियोमयोमा', 'स्त्रियों', 'मे', 'पाया', 'जाने', 'वाला', 'सबसे', 'सामान्य', 'ट्यूमर', 'है', '।'], ['करीब', '30', 'से', '40%', 'महिलाओं', 'में', 'यह', 'गाँठ', 'पाई', 'जाती', 'है', '।'], ['नियमित', 'सोनोग्राफी', 'जाँच', 'से', 'यह', 'तथ्य', 'सामने', 'आया', 'है', '।'], ['बहुत', 'से', 'मामलों', 'में', 'यह', 'कुछ', 'भी', 'लक्षण', 'उत्पन्न', 'नहीं', 'करती', 'परन्तु', 'कई', 'मामलों', 'में', 'पीड़ा', 'रक्तस्त्राव', 'और', 'बंध्यता', 'का', 'कारण', 'बनती', 'है', '।'], ['अधिकांशतः', 'यह', '30-40', 'वर्ष', 'की', 'आयु', 'की', 'महिलाओं', 'में', 'देखने', 'को', 'मिलती', 'है', '।'], ['कम', 'उम्र', 'की', 'किशोरियाँ', 'भी', 'कभी', '-', 'कभी', 'इससे', 'ग्रस्त', 'हो', 'जाती', 'हैं', '।'], ['रजोनिवृत्ति', 'होने', 'पर', 'रसौली', 'का', 'आकार', 'छोटा', 'हो', 'जाता', 'है', '।'], ['लेकिन', 'हार्मोन', 'थेरैपी', 'लेने', 'पर', 'पुनः', 'आकार', 'बड़ा', 'हो', 'सकता', 'है', '।'], ['गहरेरंग', 'वाली', 'महिलाओं', 'में', 'गोरे', 'रंग', 'वाली', 'महिलाओं', 'के', 'तुलना', 'में', 

In [0]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 32
HIDDEN_DIM = 32

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [0]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,bidirectional=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out1, _ = self.lstm(embeds.view(len(sentence),1,-1))
        tag_space = self.hidden2tag(lstm_out1.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [11]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
training_data,testing_data = train_test_split(training_data,test_size=0.2)
for epoch in range(20):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        #print(sentence,tags)
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(testing_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    tag_seq=''
    print(len(tag_scores))
    for i in range(0,len(tag_scores)):
      m=max(tag_scores[i])
      for j in range(0,len(tag_scores[i])):
        if m==tag_scores[i][j]:
          print(ix_to_tag[j],end='-')
          tag_seq+=ix_to_tag[j]+'-'
print('\n')
print(tag_seq.split('-'))
print(testing_data[0][1])

#print(tag_seq)

tensor([[-4.1687, -4.5414, -4.4407,  ..., -4.2570, -4.4730, -4.3297],
        [-4.3378, -4.4665, -4.3907,  ..., -4.3426, -4.4697, -4.3296],
        [-4.2095, -4.4799, -4.3830,  ..., -4.3441, -4.5752, -4.3206],
        ...,
        [-4.0153, -4.3367, -4.3342,  ..., -4.2591, -4.5047, -4.3309],
        [-4.1266, -4.3291, -4.3457,  ..., -4.2652, -4.4511, -4.3711],
        [-4.1829, -4.3202, -4.3136,  ..., -4.4070, -4.4607, -4.4184]])
12
DM_DMD-RP_NEG-RD_PUNC-DM_DMD-PR_PRP-RP_RPD-V_VM-N_NN-PSP-V_VM-V_VAUX-RD_PUNC-

['DM_DMD', 'RP_NEG', 'RD_PUNC', 'DM_DMD', 'PR_PRP', 'RP_RPD', 'V_VM', 'N_NN', 'PSP', 'V_VM', 'V_VAUX', 'RD_PUNC', '']
['PR_PRP', 'RP_NEG', 'RD_PUNC', 'PR_PRP', 'PR_PRP', 'N_NNP', 'N_NNP', 'N_NN', 'PSP', 'V_VM', 'V_VAUX', 'RD_PUNC']
